In [1]:
################################################################################
# CNN based Pokemon Classfier                                                  #
# ---------------------------------------------------------------------------- #
# ~600 images of 20 pokemon are used to classify them.                         #
# A simple convnet is used to classify the images.                             #
# Compared with EigenFaces on a smaller dataset.                               #
# ---------------------------------------------------------------------------- #
# NOTE: The first version of my model is put in v1_scripts directory.          #
################################################################################

%pylab inline
from PIL import Image, ImageOps
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from time import time
import os

"""
Read images as features and labels
"""
numpy_X = []
numpy_y = []
labels = []

for root, dirs, files in os.walk('dataset/'):
    labels.append(root.split('/')[1])
    for f in files:
        imgF = Image.open(os.path.join(root, f)).convert('RGB') # Open the file in RGB mode
        imgF = ImageOps.fit(image=imgF, size=(64, 64), method=Image.ANTIALIAS) # Crop the image
        
        numpy_X.append(np.array(imgF))
        numpy_y.append(root.split('/')[1])
        
# remove empty entry from labels
labels = labels[1:]

# Convert y from labels to indexes
for i in range(len(numpy_y)):
    numpy_y[i] = labels.index(numpy_y[i])

# Convert to np.array
numpy_X = np.array(numpy_X).reshape(-1, 3, 64, 64)
numpy_y = np.array(numpy_y)

print(numpy_X.shape, numpy_y[:5])
print(labels)

Populating the interactive namespace from numpy and matplotlib
(596, 3, 64, 64) [0 0 0 0 0]
['abra', 'alakazam', 'articuno', 'blastoise', 'bulbasaur', 'charizard', 'charmander', 'charmeleon', 'gengar', 'ivysaur', 'magikarp', 'meowth', 'mew', 'mewtwo', 'moltres', 'pikachu', 'squirtle', 'venusaur', 'wartortle', 'zapdos']


In [2]:
"""
Numpy to Tensors (CUDA)
"""
X = torch.from_numpy(numpy_X).float().cuda()
y = torch.from_numpy(numpy_y).cuda()

print(X.type(), y.type())

# PyTorch dataset
dataset = torch.utils.data.TensorDataset(X, y)

# Test train split
total = len(dataset)
trainSize = int(total * 0.75)
testSize = total - trainSize
train, test = torch.utils.data.random_split(dataset, (trainSize, testSize))

print("Train Size: ", len(train))
print("Test Size: ", len(test))

torch.cuda.FloatTensor torch.cuda.LongTensor
Train Size:  447
Test Size:  149


In [3]:
"""
Train Batch Loader for Batch SGD
"""
BATCH_SIZE=30
train_loader = torch.utils.data.DataLoader(train, batch_size = BATCH_SIZE, shuffle = False)

In [4]:
"""
ConvNet
"""
class SimpleConvNet(nn.Module):
    def __init__(self):
        super(SimpleConvNet, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=3)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=3)
        self.fc1 = nn.Linear(3136, 512)
        self.fc2 = nn.Linear(512, 128)
        self.fc3 = nn.Linear(128, 20)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), 2)
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))

        return x

    # copied from torch example
    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
    
device = torch.device("cuda")
cnn = SimpleConvNet().to(device)
print(cnn)

SimpleConvNet(
  (conv1): Conv2d(3, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=3136, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=20, bias=True)
)


In [5]:
"""
Training
"""
NUM_EPOCHS = 60
optimizer = optim.SGD(cnn.parameters(), lr=0.001)
error = nn.CrossEntropyLoss()

for epoch in range(NUM_EPOCHS):
    correct = 0
    for batch_idx, (X_batch, y_batch) in enumerate(train_loader):
        optimizer.zero_grad()
        output = cnn(X_batch)
        loss = error(output, y_batch)
        loss.backward()
        optimizer.step()
        
        predicted = torch.max(output.data, 1)[1]
        correct += (predicted == y_batch.cuda()).sum()
        if batch_idx % 2 == 0:
            print('Epoch : {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\t Accuracy:{:.3f}%'.format(
                    epoch, batch_idx*len(X_batch), len(train_loader.dataset), 100.*batch_idx / len(train_loader),
                    loss.data.item(), float(correct*100) / float(BATCH_SIZE*(batch_idx+1))))

Epoch : 0 [0/447 (0%)]	Loss: 3.704450	 Accuracy:6.667%
Epoch : 0 [60/447 (13%)]	Loss: 3.058440	 Accuracy:6.667%
Epoch : 0 [120/447 (27%)]	Loss: 2.985214	 Accuracy:6.000%
Epoch : 0 [180/447 (40%)]	Loss: 3.025889	 Accuracy:4.762%
Epoch : 0 [240/447 (53%)]	Loss: 3.092937	 Accuracy:4.444%
Epoch : 0 [300/447 (67%)]	Loss: 2.984516	 Accuracy:5.152%
Epoch : 0 [360/447 (80%)]	Loss: 3.135553	 Accuracy:4.872%
Epoch : 0 [378/447 (93%)]	Loss: 3.032290	 Accuracy:4.667%
Epoch : 1 [0/447 (0%)]	Loss: 2.971138	 Accuracy:6.667%
Epoch : 1 [60/447 (13%)]	Loss: 2.940642	 Accuracy:7.778%
Epoch : 1 [120/447 (27%)]	Loss: 2.943166	 Accuracy:8.000%
Epoch : 1 [180/447 (40%)]	Loss: 2.854158	 Accuracy:9.524%
Epoch : 1 [240/447 (53%)]	Loss: 3.011880	 Accuracy:7.407%
Epoch : 1 [300/447 (67%)]	Loss: 2.892198	 Accuracy:8.485%
Epoch : 1 [360/447 (80%)]	Loss: 3.017347	 Accuracy:8.462%
Epoch : 1 [378/447 (93%)]	Loss: 2.992513	 Accuracy:8.000%
Epoch : 2 [0/447 (0%)]	Loss: 2.935112	 Accuracy:13.333%
Epoch : 2 [60/447 (13%)]

Epoch : 17 [240/447 (53%)]	Loss: 2.618960	 Accuracy:30.741%
Epoch : 17 [300/447 (67%)]	Loss: 2.316410	 Accuracy:32.727%
Epoch : 17 [360/447 (80%)]	Loss: 2.355863	 Accuracy:33.077%
Epoch : 17 [378/447 (93%)]	Loss: 2.335120	 Accuracy:34.000%
Epoch : 18 [0/447 (0%)]	Loss: 2.418076	 Accuracy:33.333%
Epoch : 18 [60/447 (13%)]	Loss: 2.147155	 Accuracy:38.889%
Epoch : 18 [120/447 (27%)]	Loss: 2.498698	 Accuracy:36.667%
Epoch : 18 [180/447 (40%)]	Loss: 2.318192	 Accuracy:36.190%
Epoch : 18 [240/447 (53%)]	Loss: 2.609406	 Accuracy:34.074%
Epoch : 18 [300/447 (67%)]	Loss: 2.238877	 Accuracy:35.455%
Epoch : 18 [360/447 (80%)]	Loss: 2.318377	 Accuracy:35.385%
Epoch : 18 [378/447 (93%)]	Loss: 2.276794	 Accuracy:36.000%
Epoch : 19 [0/447 (0%)]	Loss: 2.384197	 Accuracy:36.667%
Epoch : 19 [60/447 (13%)]	Loss: 2.096321	 Accuracy:42.222%
Epoch : 19 [120/447 (27%)]	Loss: 2.457874	 Accuracy:38.000%
Epoch : 19 [180/447 (40%)]	Loss: 2.268535	 Accuracy:38.095%
Epoch : 19 [240/447 (53%)]	Loss: 2.571569	 Accur

Epoch : 37 [120/447 (27%)]	Loss: 1.815628	 Accuracy:55.333%
Epoch : 37 [180/447 (40%)]	Loss: 1.762266	 Accuracy:55.238%
Epoch : 37 [240/447 (53%)]	Loss: 1.906888	 Accuracy:54.444%
Epoch : 37 [300/447 (67%)]	Loss: 1.210874	 Accuracy:56.364%
Epoch : 37 [360/447 (80%)]	Loss: 1.577520	 Accuracy:56.154%
Epoch : 37 [378/447 (93%)]	Loss: 1.704467	 Accuracy:56.222%
Epoch : 38 [0/447 (0%)]	Loss: 1.675999	 Accuracy:56.667%
Epoch : 38 [60/447 (13%)]	Loss: 1.416735	 Accuracy:58.889%
Epoch : 38 [120/447 (27%)]	Loss: 1.829565	 Accuracy:54.667%
Epoch : 38 [180/447 (40%)]	Loss: 2.210877	 Accuracy:51.429%
Epoch : 38 [240/447 (53%)]	Loss: 1.906049	 Accuracy:50.370%
Epoch : 38 [300/447 (67%)]	Loss: 1.180189	 Accuracy:52.727%
Epoch : 38 [360/447 (80%)]	Loss: 1.541090	 Accuracy:53.077%
Epoch : 38 [378/447 (93%)]	Loss: 1.694995	 Accuracy:53.556%
Epoch : 39 [0/447 (0%)]	Loss: 1.635313	 Accuracy:56.667%
Epoch : 39 [60/447 (13%)]	Loss: 1.375350	 Accuracy:60.000%
Epoch : 39 [120/447 (27%)]	Loss: 1.739107	 Accur

Epoch : 55 [300/447 (67%)]	Loss: 0.889027	 Accuracy:60.303%
Epoch : 55 [360/447 (80%)]	Loss: 1.207752	 Accuracy:60.513%
Epoch : 55 [378/447 (93%)]	Loss: 1.330222	 Accuracy:60.444%
Epoch : 56 [0/447 (0%)]	Loss: 1.427658	 Accuracy:56.667%
Epoch : 56 [60/447 (13%)]	Loss: 1.091146	 Accuracy:63.333%
Epoch : 56 [120/447 (27%)]	Loss: 1.322795	 Accuracy:60.667%
Epoch : 56 [180/447 (40%)]	Loss: 1.494592	 Accuracy:60.476%
Epoch : 56 [240/447 (53%)]	Loss: 1.581171	 Accuracy:58.889%
Epoch : 56 [300/447 (67%)]	Loss: 0.885075	 Accuracy:60.303%
Epoch : 56 [360/447 (80%)]	Loss: 1.199935	 Accuracy:60.513%
Epoch : 56 [378/447 (93%)]	Loss: 1.320078	 Accuracy:60.444%
Epoch : 57 [0/447 (0%)]	Loss: 1.425483	 Accuracy:56.667%
Epoch : 57 [60/447 (13%)]	Loss: 1.080893	 Accuracy:63.333%
Epoch : 57 [120/447 (27%)]	Loss: 1.300220	 Accuracy:60.667%
Epoch : 57 [180/447 (40%)]	Loss: 1.483230	 Accuracy:60.476%
Epoch : 57 [240/447 (53%)]	Loss: 1.571736	 Accuracy:58.889%
Epoch : 57 [300/447 (67%)]	Loss: 0.879416	 Accur

In [6]:
correct = 0
for i in range(len(test)):
    if cnn(test[i][0].reshape(1, 3, 64, 64))[0].max(0)[1] == test[i][1]:
        correct += 1
        
print(correct / len(test))

0.18791946308724833
